<a href="https://colab.research.google.com/github/Yewon9/Emotion_Recognition_STT/blob/main/STT_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!pip install SpeechRecognition
!pip install pydub

In [13]:
import pandas as pd
from tqdm import tqdm
import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence

# DATA

In [4]:
data_emotion = pd.read_csv('data_emotion.csv')
data_main = pd.read_csv('data_main.csv')

In [5]:
data_emotion

,Emotion,Path
0,sad,/content/drive/MyDrive/project/data_aihub/5/5e...
1,sad,/content/drive/MyDrive/project/data_aihub/4/5e...
2,sad,/content/drive/MyDrive/project/data_aihub/5_2/...
3,sad,/content/drive/MyDrive/project/data_aihub/5/5f...
4,sad,/content/drive/MyDrive/project/data_aihub/5_2/...
...,...,...
12674,happy,/content/drive/MyDrive/project/data_aihub/5_2/...
12675,happy,/content/drive/MyDrive/project/data_aihub/5_2/...
12676,happy,/content/drive/MyDrive/project/data_aihub/5_2/...
12677,happy,/content/drive/MyDrive/project/data_aihub/5/5e...


In [6]:
data_main

,Path
0,/content/drive/MyDrive/project/aihub_data_main...
1,/content/drive/MyDrive/project/aihub_data_main...
2,/content/drive/MyDrive/project/aihub_data_main...
3,/content/drive/MyDrive/project/aihub_data_main...
4,/content/drive/MyDrive/project/aihub_data_main...
...,...
409,/content/drive/MyDrive/project/aihub_data_main...
410,/content/drive/MyDrive/project/aihub_data_main...
411,/content/drive/MyDrive/project/aihub_data_main...
412,/content/drive/MyDrive/project/aihub_data_main...


# STT

## emotion

In [7]:
recognizer = sr.Recognizer()

In [8]:
def speech_to_text(file_path):
    try:
        with sr.AudioFile(file_path) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(audio, language = "ko-KR")
            return text
    except sr.UnknownValueError:
        return "음성을 인식할 수 없음"
    except sr.RequestError as e:
        return f"STT API 오류: {e}"

In [9]:
data_emotion['Text'] = [speech_to_text(path) for path in tqdm(data_emotion['Path'])]

100%|██████████| 12679/12679 [2:51:52<00:00,  1.23it/s]


In [10]:
data_emotion

,Emotion,Path,Text
0,sad,/content/drive/MyDrive/project/data_aihub/5/5e...,고마워 지금 하고 올게
1,sad,/content/drive/MyDrive/project/data_aihub/4/5e...,음성을 인식할 수 없음
2,sad,/content/drive/MyDrive/project/data_aihub/5_2/...,주식을 시작했어
3,sad,/content/drive/MyDrive/project/data_aihub/5/5f...,어 그렇게 심한 건 아니고 허리 다쳐서 병원에 입원 중
4,sad,/content/drive/MyDrive/project/data_aihub/5_2/...,음악을 들으면 우울한 내 마음이 좀 가라앉으며
...,...,...,...
12674,happy,/content/drive/MyDrive/project/data_aihub/5_2/...,마음도 홀가분한 신나는 음악을 듣는 것도 좋을 거 같아
12675,happy,/content/drive/MyDrive/project/data_aihub/5_2/...,음악 듣고 스트레스 풀고 싶다
12676,happy,/content/drive/MyDrive/project/data_aihub/5_2/...,유명한 인플루언서가 진행하고 있었어 내가 거기에 신청했
12677,happy,/content/drive/MyDrive/project/data_aihub/5/5e...,그래 한번 혼자서 산책도 해 봐야겠다


In [11]:
data_emotion.to_csv('emotion_text.csv', index = False, encoding = 'utf-8-sig')

## main

### 공백

### STT - 50개씩 나눠서

In [16]:
recognizer = sr.Recognizer()

In [17]:
def split_audio_on_silence(file_path):
  sound = AudioSegment.from_wav(file_path)

  chunks = split_on_silence(sound, min_silence_len = 1000, silence_thresh = -40)
  return chunks

In [18]:
def speech_to_text(audio_chunk):
    try:
        audio_data = audio_chunk.export(format = "wav")
        with sr.AudioFile(audio_data) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(audio, language = "ko-KR")
            return text
    except sr.UnknownValueError:
        return "음성을 인식할 수 없음"
    except sr.RequestError as e:
        return f"STT API 오류: {e}"

In [19]:
def process_audio(file_path):
    chunks = split_audio_on_silence(file_path)
    text_results = []

    for i, chunk in enumerate(chunks):
        text = speech_to_text(chunk)
        text_results.append(f"Chunk {i+1}: {text}")

    return text_results

In [ ]:
data_list = []
for file_path in tqdm(data_main['Path'][0:50], desc = "Processing 1-50 files"):
    stt_results = process_audio(file_path)
    for result in stt_results:
        data_list.append({'Path': file_path, 'STT 결과': result})

Processing 1-50 files:  14%|█▍        | 7/50 [06:25<34:01, 47.47s/it]

In [ ]:
data_main = pd.DataFrame(data_list)
data_main

,Path,STT 결과
0,/content/drive/MyDrive/project/aihub_data_main...,Chunk 1: 미영
1,/content/drive/MyDrive/project/aihub_data_main...,Chunk 2: 아이
2,/content/drive/MyDrive/project/aihub_data_main...,Chunk 3: 단짝 미애와 아이는 노래하며 웁니다
3,/content/drive/MyDrive/project/aihub_data_main...,Chunk 4: 나를 위해 나란히 누워
4,/content/drive/MyDrive/project/aihub_data_main...,Chunk 5: 랑랑이 노래합니다
...,...,...
985,/content/drive/MyDrive/project/aihub_data_main...,Chunk 95: 나는 지난주 목요일에 인터넷으로 원피스를 주문했다 내일 친구 생일...
986,/content/drive/MyDrive/project/aihub_data_main...,Chunk 96: 음성을 인식할 수 없음
987,/content/drive/MyDrive/project/aihub_data_main...,Chunk 97: 음성을 인식할 수 없음
988,/content/drive/MyDrive/project/aihub_data_main...,Chunk 98: 음성을 인식할 수 없음


In [ ]:
data_main.to_csv('main_text_1.csv', index = False, encoding = 'utf-8-sig')